In [39]:
from sklearn.linear_model import LinearRegression
import itertools
from z3 import *

with open("data.txt") as file:
    data = file.read().splitlines()

def fix_data_1(line):
    p1, p2 = line.split(" @ ")
    p1 = np.array(p1.split(", ")).astype(int)
    v = np.array(p2.split(", ")).astype(int)

    p2 = p1 + v

    X = np.vstack((p1[0], p2[0]))
    y = np.vstack((p1[1], p2[1]))

    coef = LinearRegression().fit(X, y)

    return coef.coef_[0][0], coef.intercept_[0], p1, v

def fix_data_2(line):
    p1, p2 = line.split(" @ ")
    p1 = np.array(p1.split(", ")).astype(int)
    v = np.array(p2.split(", ")).astype(int)

    return p1, v

def intersection(l1, l2):

    x = (l1[1] - l2[1]) / (l2[0] - l1[0])
    y1 = l1[1] + l1[0] * x
    y2 = l2[1] + l2[0] * x

    return np.array([x, y1])#, y2

data_1 = [fix_data_1(d) for d in data]
data_2 = [fix_data_2(d) for d in data]

In [40]:
lb = 200000000000000
ub = 400000000000000

counter = 0

for comb in list(itertools.combinations(data_1, 2)):
    isec = intersection(comb[0], comb[1])

    #print(comb[1])

    if isec[0] != np.inf:
        if isec[0] >= lb and isec[0] <= ub and isec[1] >= lb and isec[1] <= ub:

            p1 = isec - comb[0][2][:-1]
            p2 = isec - comb[1][2][:-1]
            v1 = comb[0][3][:-1]
            v2 = comb[1][3][:-1]

            if (np.sign(p1) == np.sign(v1)).all() and (np.sign(p2) == np.sign(v2)).all():
                counter += 1

counter

/var/folders/qp/_881tfz13bn6k9p0wr5d3hv40000gn/T/ipykernel_70693/2437301387.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  x = (l1[1] - l2[1]) / (l2[0] - l1[0])


28266

In [41]:
start = Ints("x y z")
velocity = Ints("xv yv zv")
ts = Ints(" ".join([f't_{n}' for n in range(len(data_2))]))

s = Solver()

for n, d in enumerate(data_2):

    stone_start = d[0]
    stone_velocity = d[1]
    t = ts[n]

    s.add(t > -1)

    for i in range(3):
        s.add(stone_start[i] + stone_velocity[i] * t == start[i] + velocity[i] * t)


s.check()
result = s.model()

total = 0
for v in start:
    val = int(result[v].params()[0])
    print(v, val)
    total += val

total

x 354954946036320
y 318916597757112
z 112745502066835


786617045860267